# Advent of Code 2018 - Day 04

## Part 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read raw data
data = pd.read_csv('input/04.txt', sep='] ', 
                   engine='python',names=['date', 'action'], 
                   converters={'date':lambda t: t[6:]})

# Convert data to a datetime object
data['date'] = pd.to_datetime(data['date'], format='%m-%d %H:%M')
data.set_index('date', inplace=True)
data.sort_index(inplace=True)

# Split off ID number
data['id'] = data['action'].apply(
    lambda t: np.int64(t.split()[1][1:]) if len(t.split()) == 4 else np.nan
)

# Split off action
data['action'] = data['action'].apply(
    lambda t: t if len(t.split())==2 else ' '.join(t.split()[2:])
)

# Forward fill ID
data.fillna(method='ffill', inplace=True)

# Remove useless actions
data = data[data['action'] != 'begins shift']

In [3]:
data.head()

,action,id
date,,
1900-03-07 00:19:00,falls asleep,1291.0
1900-03-07 00:27:00,wakes up,1291.0
1900-03-08 00:09:00,falls asleep,773.0
1900-03-08 00:45:00,wakes up,773.0
1900-03-09 00:33:00,falls asleep,863.0


In [4]:
def sleep_array(day):
    tmp = np.zeros(60)
    for i in range(0, len(day.index), 2):
        tmp[day.index[i].minute:day.index[i+1].minute] = 1
    return pd.Series(tmp)

In [5]:
sleep_table = data.groupby(pd.Grouper(freq='D')).apply(sleep_array)

In [6]:
sleep_table['id'] = data.groupby(pd.Grouper(freq='D')).apply(lambda d: d['id'].unique()[0] 
                                                             if d['id'].unique().size > 0 
                                                             else 0)

In [7]:
sleep_table.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,id
date,,,,,,,,,,,,,,,,,,,,,
1900-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1291.0
1900-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,773.0
1900-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,863.0
1900-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3191.0
1900-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2791.0


In [8]:
by_id = sleep_table.groupby('id').sum(axis=1)

In [9]:
max_total_sleep = by_id.sum(axis=1).max()
id_of_max = by_id.sum(axis=1).idxmax()
most_slept_hour = np.argmax(by_id[by_id.index==id_of_max].values)

In [10]:
id_of_max * most_slept_hour

106710.0

## Part 2

In [11]:
max_min_id = by_id.max(axis=1).idxmax()

In [12]:
max_min_id * by_id.idxmax(axis=1)[max_min_id]

10491.0